In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import dump
import csv
from surprise import accuracy
from pprint import pprint

<IPython.core.display.Javascript object>

In [3]:
csv_path = os.path.join("../data/csv/reviews_cleaned_reduced_500.csv")
TextFileReader = pd.read_csv(csv_path, chunksize=1000)  # the number of rows per chunk

dfList = []
for df in TextFileReader:
    dfList.append(df)

df = pd.concat(dfList,sort=False)


<IPython.core.display.Javascript object>

In [4]:
# load beers
csv_path = os.path.join("../data/csv/beers.csv")
beers_df = pd.read_csv(csv_path)

<IPython.core.display.Javascript object>

In [5]:
beers_df = beers_df.rename(columns={'id': 'beer_id'})

<IPython.core.display.Javascript object>

In [6]:
# Lets combine the dataframe
merge_df = pd.merge(df,
                 beers_df[['beer_id', 'name', 'style', 'brewery_id']],
                 on='beer_id')

<IPython.core.display.Javascript object>

In [7]:
# sample random trainset and testset method using Pearson similarity
# test set is made of 25% of the ratings. we are looking at similarities between items (user_based=false)
reader=Reader(rating_scale=(0,5))
data = Dataset.load_from_df(merge_df[['username', 'beer_id', 'score']], reader)

trainset, testset = train_test_split(data, test_size=.25)

sim_options = {'name': 'pearson_baseline',
               'user_based': False
               }

# We'll use KNN.
algo = KNNBasic(min_k = 10, sim_options=sim_options)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.fit(trainset).test(testset)

# Then compute RMSE
accuracy.rmse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.4694
MAE:  0.3390


0.33899617725544373

<IPython.core.display.Javascript object>

In [8]:
# Five fold cross validation method to make sure our dataset is good
algo = KNNBasic(min_k = 10, sim_options=sim_options)

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4645  0.4646  0.4617  0.4644  0.4648  0.4640  0.0012  
MAE (testset)     0.3358  0.3355  0.3346  0.3352  0.3351  0.3352  0.0004  
Fit time          13.72   18.09   13.53   13.58   13.89   14.56   1.77    
Test time         30.56   21.87   19.45   19.04   19.42 

{'test_rmse': array([0.46447705, 0.46460772, 0.46167093, 0.46438483, 0.46480734]),
 'test_mae': array([0.33581497, 0.33550483, 0.3345594 , 0.33521422, 0.33505919]),
 'fit_time': (13.716271877288818,
  18.092827558517456,
  13.532569408416748,
  13.577640771865845,
  13.886734247207642),
 'test_time': (30.555680513381958,
  21.87490749359131,
  19.45203709602356,
  19.041688442230225,
  19.418246269226074)}

<IPython.core.display.Javascript object>

In [9]:
# Lets serialize and save this prediction algorithm
# Dump algorithm and reload it
dumpfile = os.path.join('../data/dump/dump_knn_pearson_500dump_file1')
dump.dump(dumpfile, predictions, algo)

<IPython.core.display.Javascript object>

In [19]:
file_name_algo = os.path.join('../data/dump/algo_knn_pearson_500dump_file')
dump.dump(file_name_algo, algo=algo)
file_name_pred = os.path.join('../data/dump/pred_knn_pearson_500dump_file')
dump.dump(file_name_pred, predictions=predictions)

<IPython.core.display.Javascript object>

In [10]:
# Code below identifes the top 10 best and worst predictions based upon code from this 
# notebook:https://nbviewer.jupyter.org/github/NicolasHug/Surprise/blob/master/examples/notebooks/KNNBasic_analysis.ipynb
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
def get_inner_ids(riid):
    inner_ids = []
    for riid in riids:
        inner_ids.append(trainset.to_inner_iid(riid))
    return inner_ids
        
    
df_predict = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df_predict['Iu'] = df_predict.uid.apply(get_Iu)
df_predict['Ui'] = df_predict.iid.apply(get_Ui)
df_predict['err'] = abs(df_predict.est - df_predict.rui)
best_predictions = df_predict.sort_values(by='err')[:10]
worst_predictions = df_predict.sort_values(by='err')[-10:]

<IPython.core.display.Javascript object>

In [11]:
# Best Predictions:
print(best_predictions)

                 uid     iid  rui  est  \
74887   DrunkyBuddha   72170  5.0  5.0   
38995    IAMTEAMPONY      92  3.0  3.0   
217274      bmcduff2     103  1.0  1.0   
34358       bmcduff2     108  1.0  1.0   
144234     100200300    5948  3.0  3.0   
13032       bmcduff2    1054  1.0  1.0   
22017      100200300    6260  3.0  3.0   
104406      bmcduff2    1341  1.0  1.0   
189487  PC_Principal  109989  5.0  5.0   
138459   IAMTEAMPONY   15758  3.0  3.0   

                                          details  Iu    Ui  err  
74887   {'actual_k': 32, 'was_impossible': False}  54  1223  0.0  
38995   {'actual_k': 28, 'was_impossible': False}  50  2251  0.0  
217274  {'actual_k': 17, 'was_impossible': False}  21  1320  0.0  
34358   {'actual_k': 17, 'was_impossible': False}  21   861  0.0  
144234  {'actual_k': 24, 'was_impossible': False}  46   493  0.0  
13032   {'actual_k': 12, 'was_impossible': False}  21   748  0.0  
22017   {'actual_k': 21, 'was_impossible': False}  46  1536  0.0  
1

<IPython.core.display.Javascript object>

In [12]:
# Worst Predictions:
print(worst_predictions)

                       uid    iid   rui       est  \
31272             YogiBeer  72720  1.00  4.324069   
219610            OzmanBey    141  1.00  4.337622   
76702             IvanJ126    332  1.00  4.355096   
80415              jmiah22    567  1.00  4.437230   
76732     Sails_Ambassador   1585  1.00  4.459066   
5142           cclark67789     92  1.00  4.486041   
65205           jallen5076  88880  1.28  4.772604   
130600       nathanjohnson  55939  1.00  4.513062   
1066                  dwaz   7520  1.24  4.782613   
121245  LloydBobBoozehound  30420  1.00  4.551155   

                                          details  Iu    Ui       err  
31272   {'actual_k': 14, 'was_impossible': False}  42   432  3.324069  
219610  {'actual_k': 23, 'was_impossible': False}  43  1526  3.337622  
76702   {'actual_k': 10, 'was_impossible': False}  23   970  3.355096  
80415   {'actual_k': 15, 'was_impossible': False}  59   641  3.437230  
76732   {'actual_k': 13, 'was_impossible': False}  27  1

<IPython.core.display.Javascript object>

In [13]:
def get_beer_name (beer_raw_id):
    beer_name = beers_df.loc[beers_df.beer_id==beer_raw_id,'name'].values[0]
    return beer_name

def get_beer_style (beer_raw_id):
    beer_style = beers_df.loc[beers_df.beer_id==beer_raw_id,'style'].values[0]
    return beer_style

def get_beer_score_mean (beer_raw_id):
    score_mean = mean_score.loc[mean_score.beer_id==beer_raw_id,'score'].values[0]
    return score_mean

def get_beer_neighbors (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                  for inner_id in beer_neighbors)
    return(beer_neighbors)

def get_beer_recc_df (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                      for inner_id in beer_neighbors)
    beers_id_recc = []
    beer_name_recc =[]
    beer_style_recc = []
    beer_score_mean = []
    for beer in beer_neighbors:
        beers_id_recc.append(beer)
        beer_name_recc.append(get_beer_name(beer))
        beer_style_recc.append(get_beer_style(beer))
        beer_score_mean.append(get_beer_score_mean(beer))
    beer_reccomendations_df = pd.DataFrame(list(zip(beers_id_recc,beer_name_recc,beer_style_recc,beer_score_mean)),
                                       columns=['beer_id', 'name', 'style', 'score_mean'])
    return beer_reccomendations_df
    
def get_inner_ids(riids):
    inner_ids = []
    for riid in riids:
        inner_ids.append(trainset.to_inner_iid(riid))
    return inner_ids
        

<IPython.core.display.Javascript object>

In [14]:
#Create mean score dataframe and get riids (riids = raw beers ids)
mean_score = merge_df.groupby('beer_id', as_index=False)[['score']].mean()
riids = mean_score['beer_id']
riids = riids.to_list()
#Get the beer inner ids
inner_ids = get_inner_ids(riids)

<IPython.core.display.Javascript object>

In [15]:
# Save the df_predict and df_ids for later use
df_predict.to_csv("../data/csv/df_predict_pearsonbaseline_500.csv",index=False)
df_ids = pd.DataFrame(list(zip(riids,inner_ids)), columns=['beer_id', 'inner_ids'])
df_ids.to_csv("../data/csv/df_ids_pearsonbaseline_500.csv",index=False)

<IPython.core.display.Javascript object>

In [16]:
# Create pickles of our dataframes to be used in the load program

filter_df = merge_df.drop(['username', 'score'], axis=1).drop_duplicates(subset =
                                                                        ['beer_id',
                                                                        'name',
                                                                        'style',
                                                                        'brewery_id'],
                                                                        keep="first")
iids_merged = pd.merge(df_ids, mean_score[['beer_id', 'score']], on='beer_id')

<IPython.core.display.Javascript object>

In [17]:
merged_iid_df = pd.merge(iids_merged,
                 filter_df[['beer_id', 'name', 'style', 'brewery_id']],
                 on='beer_id', )

<IPython.core.display.Javascript object>

In [18]:
merged_iid_df.to_pickle("../data/dump/beer.pkl")
df_predict.to_pickle('../data/dump/df_predict.pkl')

<IPython.core.display.Javascript object>